In [25]:
import numpy as np


class Martingale:
    def __init__(self, open_price, max_cnt, direction, bs, margin_call, close_call):
        self.max_cnt = max_cnt  # 最大补仓次数
        self.direction = direction  # 方向 buy 多 sell 空
        self.bs = bs  # 补仓数量 like: [1, 2, 4, 8]...
        self.margin_call = margin_call  # 跌 | 涨 1% 补仓
        self.close_call = close_call  # 获利多少平仓
        self.curr = 0  # 当前开仓次数

        self.price_lists = np.array([open_price])

        for i in range(1, max_cnt):
            if self.direction == 'buy':
                self.price_lists = np.append(
                    self.price_lists, self.price_lists[-1] - (self.price_lists[-1] * margin_call[i]))
            else:
                self.price_lists = np.append(
                    self.price_lists, self.price_lists[-1] + (self.price_lists[-1] * margin_call[i]))

    def is_add_open(self, price):  # 是否加仓
        if self.curr >= self.max_cnt:
            return []
        if self.direction == 'buy':
            return self.price_lists[self.price_lists >= price]
        else:
            return self.price_lists[self.price_lists <= price]

    def is_close_all(self, price):  # 是否平仓
        if self.curr == 0:
            return False

        result = 0
        for i in range(0, self.curr):
            result += self.price_lists[i] * self.bs[i]

        avg_price = result / sum(self.bs[:self.curr])

        if self.direction == 'buy':
            return price > (avg_price + (avg_price * self.close_call[self.curr - 1]))
        else:
            return price < (avg_price - (avg_price * self.close_call[self.curr - 1]))

    def curr_open(self):  # 当前开仓
        if self.curr == 0:
            return 0
        return sum(self.bs[:self.curr])


mas = Martingale(open_price=2000, max_cnt=5, direction='buy',
                 margin_call=0.01, close_call=0.005)

print(mas.is_add_open(1960))


[2000.  1980.  1960.2]


In [31]:
import numpy as np


max_cnt = 5
direction = 'buy'
margin_call = np.linspace(0, 0.01, num=5).tolist()
close_call = np.linspace(0.02, 0.01, num=5).tolist()
bs = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192]
price_lists = np.array([2000])
close_lists = []
order_lists = []

for i in range(1, max_cnt):
    if direction == 'buy':
        price_lists = np.append(
            price_lists, price_lists[-1] - (price_lists[-1] * margin_call[i]))
    else:
        price_lists = np.append(
            price_lists, price_lists[-1] + (price_lists[-1] * margin_call[i]))


result = 0
for i in range(0, max_cnt):
    result += price_lists[i] * bs[i]
    avg = result / sum(bs[:i+1])
    if direction == 'buy':
        close_lists.append(avg + (avg * close_call[i]))
    else:
        close_lists.append(avg - (avg * close_call[i]))
    
print(price_lists)
print(close_lists)


for item in price_lists.round(2):
    print(item) 





[2000.         1995.         1985.025      1970.1373125  1950.43593938]
[2040.0, 2031.6083333333333, 2019.8645, 2004.1558987499998, 1984.1017827516127]
2000.0
1995.0
1985.02
1970.14
1950.44


In [16]:
import time
import numpy as np
from threading import Timer
from huobi.linear_swap.rest import account, market, order, trigger_order
from config.linairx001 import ACCESS_KEY, SECRET_KEY

symbol = 'DOGE-USDT'

client = market.Market()
orderClient = order.Order(ACCESS_KEY,
                          SECRET_KEY)
triggerOrderClient = trigger_order.TriggerOrder(ACCESS_KEY,
                          SECRET_KEY)
accountClient = account.Account(ACCESS_KEY, SECRET_KEY)

result = accountClient.get_balance_valuation({"valuation_asset": 'USD'})
# result = accountClient.get_account_info({"contract_code": 'XRP-USD'})
# result = accountClient.get_position_info({"contract_code": 'XRP-USD'})


# 价格精度 数值
result = client.get_contract_info({"contract_code": symbol})
price_tick = str(result.get('data')[0].get('price_tick'))

if '1e-' in price_tick:
    print(int(price_tick.split('-')[1]))
else:
    print(len(price_tick.split('.')[1]))


def get_contract_info(): # 获取合约信息和杠杆倍数
    contract_info = accountClient.cross_get_available_level_rate({})
    result = []
    if not contract_info == None and len(contract_info.get('data')) > 0:
        for item in contract_info.get('data'):
            result.append({
                "symbol": item.get('contract_code').split('-')[0].lower(),
                "level_rate": item.get('available_level_rate').split(',')[-1]
            })
    print(result)
    return result


def fetchKLines(symbol: str, interval: str, limit: str):
    return client.get_kline(
        {"contract_code": symbol, "period": interval, "size": limit})


result = fetchKLines(symbol=symbol, interval='15min', limit='96')

# print(result)
max_index = 0
min_index = 0
klines = []

if not result == None and len(result.get('data')) > 0:
    klines = result.get('data')

    print(f"当前k线len {len(klines)}")
    for i in range(1, len(klines)):
        max_index = max_index if klines[max_index].get(
            'high') > klines[i].get('high') else i
        min_index = min_index if klines[min_index].get(
            'low') < klines[i].get('low') else i

print(f"max: {klines[max_index]}, min: {klines[min_index]}")
print(f"max_index: {max_index}, min_index: {min_index}")

high = klines[max_index].get('high')
low = klines[min_index].get('low')
middle = ((high - low) / 2) + low

change = ((high - low) / low) * 100

print(f"change: {change}")
print(f"middle: {middle}")
print(f"last: {klines[-1]}")

get_contract_info()

if change > 5:
    if max_index - min_index >= 6:  # 最大值和最小值的间隔必须要大于6根k线，过滤急拉急跌的行情
        maxv = max_index
        minv = max_index
        if len(klines) == max_index + 1:
            print('当前k线就是最大值，看之前5根k线量是否递增，是的话直接买入，止损挂在low的位置')
        for i in range(max_index + 1, len(klines)):
            # print(klines[i])
            maxv = maxv if klines[maxv].get(
                'high') > klines[i].get('high') else i
            minv = minv if klines[minv].get(
                'low') < klines[i].get('low') else i
        
        print(f"maxv: {maxv}, minv: {minv}")
        print(f"maxv_kline: {klines[maxv]}, minv_kline: {klines[minv]}")
        se_high = klines[maxv].get('high')
        se_low = klines[minv].get('low')
        se_change = ((se_high - se_low) / se_low) * 100
        print(f"se_change: {se_change}")

        if se_change <= 5:
            print('次级振幅小于5%， 开启多头马丁')
        else:
            print('次级振幅大于5%， 关闭多头马丁')

elif change <= 5:
    print('震荡行情，关闭多/空马丁')


# 计划委托下单
# result = triggerOrderClient.cross_order(data={
#     "contract_code": symbol,
#     "volume": 1,
#     "direction": 'buy',
#     "offset": 'open',
#     "trigger_type": 'le',
#     "trigger_price": '0.00001040',
#     # "order_price": '0.00001020',
#     "lever_rate": 10,
#     "order_price_type": 'optimal_20'
# })

# 查询当前计划委托
result = triggerOrderClient.cross_get_open_orders({
    "contract_code": symbol,
})

print(result)

# 查询历史计划委托
result = triggerOrderClient.cross_get_his_orders({
    "contract_code": symbol,
    "trade_type": 0,
    "status": '0',
    "create_date": 1
})

format(float('1.048e-05'), '.8f')
# 1035520928496332800
# 1035521868078092290
print(result)


6
当前k线len 96
max: {'id': 1667057400, 'open': 0.120803, 'close': 0.1325, 'low': 0.120276, 'high': 0.147287, 'amount': 241209200, 'vol': 2412092, 'trade_turnover': 31987436.0246, 'count': 12775}, min: {'id': 1666983600, 'open': 0.083879, 'close': 0.082845, 'low': 0.08239, 'high': 0.083987, 'amount': 21421400, 'vol': 214214, 'trade_turnover': 1775958.653, 'count': 1387}
max_index: 89, min_index: 7
change: 78.76805437553101
middle: 0.11483850000000001
last: {'id': 1667062800, 'open': 0.134312, 'close': 0.138709, 'low': 0.1331, 'high': 0.14, 'amount': 31987400, 'vol': 319874, 'trade_turnover': 4377046.0746, 'count': 2095}
[{'symbol': 'mana', 'level_rate': '75'}, {'symbol': 'nkn', 'level_rate': '20'}, {'symbol': 'chz', 'level_rate': '75'}, {'symbol': 'dot', 'level_rate': '75'}, {'symbol': 'okb', 'level_rate': '20'}, {'symbol': 'knc', 'level_rate': '50'}, {'symbol': 'mina', 'level_rate': '20'}, {'symbol': 'eos', 'level_rate': '75'}, {'symbol': 'xtz', 'level_rate': '50'}, {'symbol': 'luna', 'l

In [7]:
a = {'a': 1}

print(a.get('b'))

None


In [41]:
for i in range(1, 15):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [53]:
result = orderClient.cross_cancel({
    "order_id": '1031209124452663299',
    "contract_code": 'CLV-USDT'
})

print(result)
result = orderClient.cross_get_order_info({
    "order_id": '1031209124452663299',
    "contract_code": 'CLV-USDT'
})

print(result)

{'status': 'ok', 'data': {'errors': [{'order_id': '1031209124452663299', 'err_code': 1071, 'err_msg': 'Repeated cancellation. Your order has been canceled.'}], 'successes': ''}, 'ts': 1665906508940}
{'status': 'ok', 'data': [{'business_type': 'swap', 'contract_type': 'swap', 'pair': 'CLV-USDT', 'symbol': 'CLV', 'contract_code': 'CLV-USDT', 'volume': 1, 'price': 0.09, 'order_price_type': 'limit', 'order_type': 1, 'direction': 'buy', 'offset': 'open', 'lever_rate': 10, 'order_id': 1031209124452663299, 'client_order_id': None, 'created_at': 1665901014203, 'trade_volume': 0, 'trade_turnover': 0, 'fee': 0, 'trade_avg_price': None, 'margin_frozen': 0.0, 'profit': 0, 'status': 7, 'order_source': 'api', 'order_id_str': '1031209124452663299', 'fee_asset': 'USDT', 'liquidation_type': '0', 'canceled_at': 1665901034528, 'margin_asset': 'USDT', 'margin_account': 'USDT', 'margin_mode': 'cross', 'is_tpsl': 0, 'real_profit': 0, 'trade_partition': 'USDT', 'reduce_only': 0}], 'ts': 1665906509054}


In [5]:
a = 3

if not a == None and a > 2:
    print(23)

23


In [12]:
symbols = [{
    "symbol": 'btc',
    "lever_rate": '200'
}, {
    "symbol": 'eth',
    "lever_rate": '200'
}, {
    "symbol": 'clv',
    "lever_rate": '20'
}, {
    "symbol": 'dydx',
    "lever_rate": '50'
}, {
    "symbol": 'ht',
    "lever_rate": '20'
}, {
    "symbol": 'apt',
    "lever_rate": '10'
}, ]

for item in symbols:
    symbol, lever_rate = item.values()
    print(symbol, lever_rate)

btc 200
eth 200
clv 20
dydx 50
ht 20
apt 10


In [8]:
import subprocess
import json

out = subprocess.run('pm2 jlist', stdout=subprocess.PIPE, shell=True)

for item in json.loads(out.stdout):
    print(item.get('name'), item.get('pm2_env').get('status'))




In [5]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964,
  "year": 2020
}
thisdict
# thisdict['aaa'] = 345
# thisdict
float('-inf')


-inf

In [3]:
i = 'sell'

'sell' if i == 'buy' else 'buy'

3 / 4 <= 1/2

False

In [52]:


np.linspace(0.01, 0, num=5)

# [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192][:5]

array([0.01  , 0.0075, 0.005 , 0.0025, 0.    ])

In [2]:
import numpy as np

np.linspace(0.01, -0.01, num=5).tolist()

[0.01, 0.005, 0.0, -0.004999999999999999, -0.01]

In [1]:
import numpy as np
print(np.linspace(0, 0.01, num=5).tolist())
print(np.linspace(0.05, 0.01, num=5).tolist())

[0.0, 0.0025, 0.005, 0.0075, 0.01]
[0.05, 0.04, 0.030000000000000002, 0.020000000000000004, 0.01]


In [19]:
[float(item) for item in "123.3,2,2,2".split(',')]

[123.3, 2.0, 2.0, 2.0]

In [3]:
import math

math.floor(199.123123 / 100) * 4

'SECRET_KEY'.lower()

'secret_key'

In [58]:
curr = 0
ss = 1

def a():
    global curr, ss 
    curr = 1
    ss = 2

a()
print(curr, ss)

1 2


In [2]:
'start main.py --name clv_buy --interpreter python3 -- --symbol CLV --max_cnt 5 --direction buy --margin_call "0.0,0.0025,0.005,0.0075,0.01" --close_call "0.05,0.04,0.03,0.02,0.01"'.split(' ')

['start',
 'main.py',
 '--name',
 'clv_buy',
 '--interpreter',
 'python3',
 '--',
 '--symbol',
 'CLV',
 '--max_cnt',
 '5',
 '--direction',
 'buy',
 '--margin_call',
 '"0.0,0.0025,0.005,0.0075,0.01"',
 '--close_call',
 '"0.05,0.04,0.03,0.02,0.01"']